In [2]:
import os
import pandas as pd
from SigProfilerAssignment import Analyzer as Analyze
import matplotlib.pyplot as plt
import gc
import re

In [3]:
os.getcwd()

'd:\\Projects\\黃怡菁'

In [3]:
tmp=pd.read_csv("somatic_variant_v1.tsv",sep="\t")

In [4]:
tmp.head()

,Chr,Start,End,Ref,Alt,tumor,id,Func.refGeneWithVer,Gene.refGeneWithVer,GeneDetail.refGeneWithVer,ExonicFunc.refGeneWithVer,AAChange.refGeneWithVer,AF,AF_eas,TaiwanBioBank,Variant_type
0,chr1,11168338,11168338,C,A,0.30067,4022266,exonic,MTOR,.,nonsynonymous SNV,MTOR:NM_004958.4:exon57:c.7534G>T:p.D2512Y,.,.,.,SNV
1,chr1,11174416,11174416,G,A,0.33423,7221871,exonic,MTOR,.,nonsynonymous SNV,MTOR:NM_004958.4:exon53:c.7259C>T:p.A2420V,.,.,.,SNV
2,chr1,11174508,11174508,A,C,0.09275,20156998,exonic,MTOR,.,synonymous SNV,MTOR:NM_004958.4:exon53:c.7167T>G:p.V2389V,.,.,.,SNV
3,chr1,11174915,11174915,C,T,0.01179,267273,exonic,MTOR,.,synonymous SNV,MTOR:NM_004958.4:exon52:c.7119G>A:p.E2373E,.,.,.,SNV
4,chr1,11181417,11181417,C,T,0.04202,20074234,exonic,MTOR,.,synonymous SNV,MTOR:NM_004958.4:exon49:c.6819G>A:p.P2273P,0.0002,0,.,SNV


In [5]:
aa=tmp['id'].value_counts()
aa=pd.DataFrame.from_dict({"sample_id":aa.index,"variant_count":aa.values})

In [6]:
aa

,sample_id,variant_count
0,267273,11122
1,1680596,1497
2,8866572,829
3,20156998,580
4,9068923,355
...,...,...
222,5613371,5
223,11045151,5
224,10139366,3
225,9671689,3


In [7]:
aa.to_csv("variant_per_sample_v1.tsv",sep="\t",index=None)

In [8]:
aa['variant_count'].describe()

count      227.000000
mean       130.105727
std        745.107593
min          3.000000
25%         16.000000
50%         24.000000
75%        118.500000
max      11122.000000
Name: variant_count, dtype: float64

In [9]:
samples=tmp['id'].unique()

#### Looped mutational signature analysis

In [23]:
for sample in samples:
    sub_variants=tmp.loc[(tmp['id']==sample) & (tmp['Variant_type']=="SNV") & (tmp['tumor']>=0.02) ,['Chr','Start','id','Ref','Alt']]
    result_path=f"signature_v3/{sample}"
    if not os.path.exists(result_path):
        os.mkdir(result_path)
    sub_variants[['Chr','Start','id','Ref','Alt']].to_csv(f"{result_path}/filtered.vcf",sep="\t",index=None,header=None)
    Analyze.cosmic_fit(samples=result_path, 
                   output=f"{result_path}/Assignment",
                   input_type="vcf",
                   context_type="96",
                   genome_build="GRCh37",
                   make_plots=True,
                   sample_reconstruction_plots=True,
                   exclude_signature_subgroups=None,
                   cosmic_version=3.4)
    aetiology=pd.read_csv('aetiology_map.tsv',sep='\t')
    tmp_signature_assignment=pd.read_csv(f"{result_path}/Assignment/Assignment_Solution/Activities/Assignment_Solution_Activities.txt",sep='\t')
    tmp_signature_assignment={tmp_signature_assignment.columns[i]:tmp_signature_assignment.iloc[0,i] for i in range(1,tmp_signature_assignment.shape[1]) }
    tmp_signature_assignment={i:round(tmp_signature_assignment[i]/sum(tmp_signature_assignment.values()),2)  for i in tmp_signature_assignment.keys()}
    plotdata=pd.DataFrame.from_dict({'signature':tmp_signature_assignment.keys(),'freq':tmp_signature_assignment.values()})
    plotdata=plotdata[plotdata['freq']!=0]
    plotdata=pd.merge(plotdata,aetiology,on='signature',how='left')
    plotdata.loc[plotdata['aetiology'].isnull(),'aetiology']='Possible sequencing artefact'
    labels = list(tmp_signature_assignment.keys())
    sizes = list(tmp_signature_assignment.values())
    fig1, ax1 = plt.subplots()
    ax1.pie(plotdata['freq'], labels=plotdata['signature']+'\n'+plotdata['aetiology'], autopct='%1.1f%%', startangle=90)
    plt.savefig(f"{result_path}/pie_chart.pdf", format="pdf", bbox_inches="tight")
    plt.show()

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 1.72 seconds.
Matrices generated for 1 samples with 0 errors. Total of 22 SNVs, 0 DINUCs, and 0 INDELs were successfully analyzed.
Assigning COSMIC sigs or Signature Database ...... 
|████████████████████████████████████████| 1/1 [100%] in 0.9s (1.11/s) 


 
Your Job Is Successfully Completed! Thank You For Using SigProfilerAssignment.
 
Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 1.81 seconds.
Matrices generated for 1 samples with 0 errors. Total of 23 SNVs, 0 DINUCs, and 0 INDELs were successfully analyzed.
Assigning COSMIC sigs or Signature Database ...... 
|████████████████████████████████████████| 1/1 [100%] in 0.5s (2.19/s) 


 
Your Job Is Successfully Completed! Thank You For Using SigProfilerAssignment.
 
Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.23 seconds.
Matrices generated for 1 samples with 0 errors. Total of 537 SNVs, 0 DINUCs, and 0 I

### Merge all signatures

In [107]:
sample_full=pd.read_csv("variant_per_sample_v1.tsv",sep="\t")
sig_matrix=pd.DataFrame()
for sample in sample_full['sample_id']:
    result_path=f"signature_v3/{sample}"
    tmp_signature_assignment=pd.read_csv(f"{result_path}/Assignment/Assignment_Solution/Activities/Assignment_Solution_Activities.txt",sep='\t')
    tmp_signature_assignment['Samples']=sample
    total=tmp_signature_assignment.iloc[:,1:].values.sum()
    for signature in tmp_signature_assignment.columns[tmp_signature_assignment.columns.str.contains('SBS')]:
        tmp_signature_assignment[signature]=tmp_signature_assignment[signature]/total
    sig_matrix=pd.concat([sig_matrix,tmp_signature_assignment],axis=0)

In [108]:
sig_matrix.head()

,Samples,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,SBS7b,SBS7c,...,SBS90,SBS91,SBS92,SBS93,SBS94,SBS95,SBS96,SBS97,SBS98,SBS99
0,267273,0.094288,0.0,0.0,0.0,0.408367,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162832,0.0
0,1680596,0.000000,0.0,0.0,0.0,0.000000,0.322359,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,8866572,0.029268,0.0,0.0,0.0,0.103659,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,20156998,0.027933,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,9068923,0.090253,0.0,0.0,0.0,0.256318,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [109]:
merged_results=pd.merge(sample_full,sig_matrix,left_on="sample_id",right_on="Samples",how="inner")

In [110]:
merged_results.head()

,sample_id,variant_count,Samples,SBS1,SBS2,SBS3,SBS4,SBS5,SBS6,SBS7a,...,SBS90,SBS91,SBS92,SBS93,SBS94,SBS95,SBS96,SBS97,SBS98,SBS99
0,267273,11122,267273,0.094288,0.0,0.0,0.0,0.408367,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162832,0.0
1,1680596,1497,1680596,0.000000,0.0,0.0,0.0,0.000000,0.322359,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,8866572,829,8866572,0.029268,0.0,0.0,0.0,0.103659,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,20156998,580,20156998,0.027933,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,9068923,355,9068923,0.090253,0.0,0.0,0.0,0.256318,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


#### Summarize QC metrics for mutational signature

In [112]:
sig_qc_table=pd.DataFrame()
for sample in sample_full.sample_id:
    result_path=f"signature_v3/{sample}"
    tmp_signature_assignment_quality=pd.read_csv(f"{result_path}/Assignment/Assignment_Solution/Solution_Stats/Assignment_Solution_Samples_Stats.txt",sep='\t')
    tmp_signature_assignment_quality['Sample Names']=sample
    sig_qc_table = pd.concat([sig_qc_table,tmp_signature_assignment_quality],axis=0)

In [113]:
sig_qc_table.head()

,Sample Names,Total Mutations,Cosine Similarity,L1 Norm,L1_Norm_%,L2 Norm,L2_Norm_%,KL Divergence,Correlation
0,267273,6215,0.951,2038.732,32.803%,267.355,31.157%,0.10359,0.891
0,1680596,1458,0.939,768.331,52.698%,149.800,37.694%,0.24802,0.933
0,8866572,820,0.986,214.891,26.206%,39.273,16.949%,0.12057,0.985
0,20156998,537,0.992,129.253,24.069%,23.311,12.895%,0.12270,0.991
0,9068923,277,0.943,131.753,47.564%,19.038,33.39%,0.29713,0.924


#### Combine all metrics

In [114]:
merged_results=pd.merge(sample_full,sig_qc_table,left_on='sample_id',right_on='Sample Names',how="inner")
merged_results=merged_results.merge(sig_matrix,left_on="sample_id",right_on="Samples",how="inner")
merged_results=merged_results.drop(columns=['Sample Names','Samples'])
merged_results=merged_results.rename(columns={'sample_id':'Sample ID','variant_count':'Variant count','Total Mutations':'Total SNV'})

In [115]:
merged_results['Sample ID']=merged_results['Sample ID'].astype(str).str.zfill(8)

In [116]:
merged_results.to_excel('merged_mutational_signatures_v3.xlsx',index=None,sheet_name="full metrics")